In [ ]:
# Загрузка библиотек
import numpy as np # для работы с массивами
import pandas as pd # для работы с DataFrame 
from sklearn import datasets # для импорта данных
import seaborn as sns # для визуализации статистических данных
import matplotlib.pyplot as plt # для построения графиков


# загружаем датасет
# Примечание. В scikit-learn 1.2 датасет о домах в Бостоне был удалён из репозитория библиотеки
# boston = datasets.load_boston()
# boston_data = pd.DataFrame(
#     data=boston.data, #данные
#     columns=boston.feature_names #наименования столбцов
# )
# boston_data['PRICE'] = boston.target
# boston_data.head()

# загружаем датасет
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE']
boston_data = pd.read_csv('data/housing.csv', header=None, delimiter=r"\s+", names=column_names)
boston_data.head()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [3]:
# составляем матрицу А и вектор целевой переменной
CRIM = boston_data['CRIM']
RM = boston_data['RM']
A = np.column_stack((np.ones(506), CRIM, RM))
y = boston_data[['PRICE']]
print(A)

[[1.0000e+00 6.3200e-03 6.5750e+00]
 [1.0000e+00 2.7310e-02 6.4210e+00]
 [1.0000e+00 2.7290e-02 7.1850e+00]
 ...
 [1.0000e+00 6.0760e-02 6.9760e+00]
 [1.0000e+00 1.0959e-01 6.7940e+00]
 [1.0000e+00 4.7410e-02 6.0300e+00]]


In [4]:
# проверим размерность
print(A.shape)
## (506, 3)

(506, 3)


In [5]:
# вычислим OLS-оценку для коэффициентов
w_hat = np.linalg.inv(A.T@A)@A.T@y
print(w_hat.values)

[[-29.24471945]
 [ -0.26491325]
 [  8.39106825]]


In [6]:
# добавились новые данные:
CRIM_new = 0.1
RM_new = 8
# делаем прогноз типичной стоимости дома
PRICE_new = w_hat.iloc[0]+w_hat.iloc[1]*CRIM_new+w_hat.iloc[2]*RM_new
print(PRICE_new.values)
## [37.85733519]

[37.85733519]


Для удобства дальнейшего использования оформим характеристики нового наблюдения в виде матрицы размером (1,3):

In [7]:
# короткий способ сделать прогноз
new=np.array([[1,CRIM_new,RM_new]])
print('prediction:', (new@w_hat).values)
## prediction: [[37.85733519]]

prediction: [[37.85733519]]


Примечание. Обратите внимание, что, решая задачу с помощью Python, мы получили немного другой результат прогноза стоимости. Это связано с тем, что при выполнении ручного расчёта мы округлили значения коэффициентов и получили менее точный результат.

Мы уже знаем, что алгоритм построения модели линейной регрессии по МНК реализован в классе LinearRegression, находящемся в модуле sklearn.linear_model. Для вычисления коэффициентов (обучения модели) нам достаточно передать в метод fit() нашу матрицу с наблюдениями и вектор целевой переменной, а для построения прогноза — вызвать метод predict():

In [8]:
from sklearn.linear_model import LinearRegression
# создаём модель линейной регрессии
model = LinearRegression(fit_intercept=False)
# вычисляем коэффициенты регрессии
model.fit(A, y)
print('w_hat:', model.coef_)
new_prediction = model.predict(new)
print('prediction:', new_prediction)
## w_hat: [[-29.24471945  -0.26491325   8.39106825]]
## prediction: [[37.85733519]]

w_hat: [[-29.24471945  -0.26491325   8.39106825]]
prediction: [[37.85733519]]


Примечание. Здесь при создании объекта класса LinearRegression мы указали fit_intercept=False, так как в нашей матрице наблюдений A **уже присутствует столбец с единицами** для умножения на свободный член w0. Его повторное добавление не имеет смысла.

### Задание 3.5

1 point possible (graded)

Сделайте прогноз типичной стоимости (в тыс. долларов) дома в городе с уровнем преступности CRIM=0.2 и средним количеством комнат в доме RM=6. В качестве модели используйте линейную регрессию, оценка вектора коэффициентов которой равна:

w⃗ ^=(−29.3,−0.26, 8.4)

21.

In [9]:
CRIM_new2 = 0.2
RM_new2 = 6
# делаем прогноз типичной стоимости дома
PRICE_new2 = w_hat.iloc[0]+w_hat.iloc[1]*CRIM_new2+w_hat.iloc[2]*RM_new2
print(PRICE_new2.values)
## [21.04870738]

[21.04870738]


In [18]:
# короткий способ сделать прогноз
new2=np.array([[1,CRIM_new2,RM_new2]])
print('prediction:', (new2@w_hat).values)
PRICE_new2_ = np.round(((new2@w_hat).values)[0], 0)
print('prediction:', PRICE_new2_)
## prediction: [[21.04870738]]
## prediction: [21.]

prediction: [[21.04870738]]
prediction: [21.]


Задание 3.7
1/1 point (graded)

Можно ли найти классическую OLS-оценку (не используя сингулярное разложение) в модели целевой переменной  y  и регрессорами  x1 ,  x2 ,  x3 ?
![alt text](2025-04-04_12-02-30.png)


In [19]:
import numpy as np
from fractions import Fraction

# Определяем данные
y = np.array([1, 2, 1, 2])
x1 = np.array([1, 1, 0, 2])
x2 = np.array([-1, 1, 0, 0])
x3 = np.array([0, 2, 0, 2])

# Формируем матрицу X (добавляем столбец из единиц для beta_0)
X = np.column_stack((np.ones(4), x1, x2, x3))

# Вычисляем X^T X
XTX = X.T @ X
print("X^T X:")
print(XTX)

# Вычисляем X^T y
XTy = X.T @ y
print("\nX^T y:")
print(XTy)

# Вычисляем OLS-оценку beta = (X^T X)^(-1) * (X^T y)
beta = np.linalg.inv(XTX) @ XTy
print("\nOLS-оценка beta (численно):")
print(beta)

# Преобразуем в дроби для удобства чтения
beta_fractions = [Fraction(x).limit_denominator() for x in beta]
print("\nOLS-оценка beta в дробях:")
print(f"beta_0 = {beta_fractions[0]}")
print(f"beta_1 = {beta_fractions[1]}")
print(f"beta_2 = {beta_fractions[2]}")
print(f"beta_3 = {beta_fractions[3]}")

X^T X:
[[4. 4. 0. 4.]
 [4. 6. 0. 6.]
 [0. 0. 2. 2.]
 [4. 6. 2. 8.]]

X^T y:
[6. 7. 1. 8.]


LinAlgError: Singular matrix

После проверки становится ясно, что столбцы ( X ) линейно зависимы, и $rank(X)<4$
, что делает $(X^{T}X)^{−1} $ вырожденной.
Решение проблемы
Поскольку $(X^{T}X)^{−1} $
 вырожденная, мы не можем использовать формулу  $(X^{T}X)^{−1}X^{T}y$
. Вместо этого используем функцию np.linalg.lstsq, которая решает задачу наименьших квадратов даже для вырожденных матриц, применяя псевдообратную матрицу (метод Мура-Пенроуза).



In [20]:
import numpy as np
from fractions import Fraction

# Определяем данные
y = np.array([1, 2, 1, 2])
x1 = np.array([1, 1, 0, 2])
x2 = np.array([-1, 1, 0, 0])
x3 = np.array([0, 2, 0, 2])

# Формируем матрицу X (добавляем столбец из единиц для beta_0)
X = np.column_stack((np.ones(4), x1, x2, x3))

# Вычисляем X^T X
XTX = X.T @ X
print("X^T X:");
print(XTX)

# Вычисляем X^T y
XTy = X.T @ y
print("\nX^T y:")
print(XTy)

# Проверяем ранг X^T X
rank_XTX = np.linalg.matrix_rank(XTX)
print(f"\nРанг X^T X: {rank_XTX}")

# Поскольку X^T X вырожденная, используем np.linalg.lstsq
beta, residuals, rank, s = np.linalg.lstsq(X, y, rcond=None)
print("\nOLS-оценка beta (численно):")
print(beta)

# Преобразуем в дроби для удобства чтения
beta_fractions = [Fraction(x).limit_denominator() for x in beta]
print("\nOLS-оценка beta в дробях:")
print(f"beta_0 = {beta_fractions[0]}")
print(f"beta_1 = {beta_fractions[1]}")
print(f"beta_2 = {beta_fractions[2]}")
print(f"beta_3 = {beta_fractions[3]}")

X^T X:
[[4. 4. 0. 4.]
 [4. 6. 0. 6.]
 [0. 0. 2. 2.]
 [4. 6. 2. 8.]]

X^T y:
[6. 7. 1. 8.]

Ранг X^T X: 3

OLS-оценка beta (численно):
[1.         0.16666667 0.16666667 0.33333333]

OLS-оценка beta в дробях:
beta_0 = 1
beta_1 = 1/6
beta_2 = 1/6
beta_3 = 1/3


# 4. Стандартизация векторов и матрица корреляций

In [21]:
boston_data[['CHAS', 'LSTAT', 'CRIM','RM']].describe()

,CHAS,LSTAT,CRIM,RM
count,506.000000,506.000000,506.000000,506.000000
mean,0.069170,12.653063,3.613524,6.284634
std,0.253994,7.141062,8.601545,0.702617
min,0.000000,1.730000,0.006320,3.561000
25%,0.000000,6.950000,0.082045,5.885500
50%,0.000000,11.360000,0.256510,6.208500
75%,0.000000,16.955000,3.677083,6.623500
max,1.000000,37.970000,88.976200,8.780000


Видим, что каждый из признаков измеряется в различных единицах и изменяется в различных диапазонах: например, CHAS лежит в диапазоне от 0 до 1, а вот CRIM — в диапазоне от 0.006 до 88.976.

Рассмотрим модель линейной регрессии по МНК без стандартизации. Помним, что необходимо добавить столбец из единиц:

In [22]:
# составляем матрицу наблюдений и вектор целевой переменной
A = np.column_stack((np.ones(506), boston_data[['CHAS', 'LSTAT', 'CRIM','RM']]))
y = boston_data[['PRICE']]
# вычисляем OLS-оценку для коэффициентов без стандартизации
w_hat=np.linalg.inv(A.T@A)@A.T@y
print(w_hat.values)

[[-1.92052548]
 [ 3.9975594 ]
 [-0.58240212]
 [-0.09739445]
 [ 5.07554248]]


Давайте вспомним интерпретацию коэффициентов построенной модели линейной регрессии, которую мы изучали в модуле «ML-2. Обучение с учителем: регрессия». Значение коэффициента w^i означает, на сколько в среднем изменится медианная цена (в тысячах долларов) при увеличении xi на 1.

Например, если количество низкостатусного населения (LSTAT) увеличится на 1 %, то медианная цена домов в районе (в среднем) упадёт на 0.6 тысяч долларов. А если среднее количество комнат (RM) в районе станет больше на 1, то медианная стоимость домов в районе (в среднем) увеличится на 5 тысяч долларов. 
Тут в голову может прийти мысль: судя по значению коэффициентов, количество комнат (RM) оказывает на стоимость жилья большее влияние, чем процент низкостатусного населения (LSTAT). Однако такой вывод будет ошибочным. Мы не учитываем, что признаки, а значит и коэффициенты линейной регрессии, лежат в разных масштабах. Чтобы говорить о важности влияния признаков на модель, нужно строить её на стандартизированных данных.

Помним, что для построения стандартизированной линейной регрессии нам не нужен вектор свободных коэффициентов, а значит и столбец из единиц тоже не понадобится.

Сначала центрируем векторы, которые находятся в столбцах матрицы A. Для этого вычтем среднее, вычисленное по строкам матрицы A в каждом столбце, с помощью метода mean(). Затем разделим результат на длины центрированных векторов, вычисленных с помощью функции linalg.norm().

Примечание. Обратите внимание, что для функции linalg.norm() обязательно необходимо указать параметр axis=0, так как по умолчанию норма считается для всей матрицы, а не для каждого столбца в отдельности. С определением нормы матрицы и тем, как она считается, вы можете ознакомиться в документации к функции norm().

In [23]:
# составляем матрицу наблюдений без дополнительного столбца из единиц
A = boston_data[['CHAS', 'LSTAT', 'CRIM','RM']]
y = boston_data[['PRICE']]
# стандартизируем векторы в столбцах матрицы A
A_cent = A - A.mean(axis=0)
A_st = A_cent/np.linalg.norm(A_cent, axis=0)
A_st.describe().round(2)

,CHAS,LSTAT,CRIM,RM
count,506.00,506.00,506.00,506.00
mean,-0.00,-0.00,-0.00,-0.00
std,0.04,0.04,0.04,0.04
min,-0.01,-0.07,-0.02,-0.17
25%,-0.01,-0.04,-0.02,-0.03
50%,-0.01,-0.01,-0.02,-0.00
75%,-0.01,0.03,0.00,0.02
max,0.16,0.16,0.44,0.16


Теперь векторы имеют одинаковые средние значения и стандартные отклонения. Если вычислить длину каждого из векторов, мы увидим, что они будут равны 1:

In [24]:
print(np.linalg.norm(A_st, axis=0))
## [1. 1. 1. 1.]

[1. 1. 1. 1.]


Для получения стандартизированных коэффициентов нам также понадобится стандартизация целевой переменной y по тому же принципу:

In [25]:
# стандартизируем вектор целевой переменной
y_cent = y - y.mean()
y_st = y_cent/np.linalg.norm(y_cent)

Формула для вычисления коэффициента та же, что и раньше, только матрица A теперь заменяется на Ast, а y — на yst:

In [26]:
# вычислим OLS-оценку для стандартизированных коэффициентов
w_hat_st=np.linalg.inv(A_st.T@A_st)@A_st.T@y_st
print(w_hat_st.values)

[[ 0.11039956]
 [-0.45220423]
 [-0.09108766]
 [ 0.38774848]]


Вновь смотрим на коэффициенты. Помним, что коэффициента w^0 у нас больше нет:

w^CHAS=0.11

w^LSTAT, st=−0.45

w^CRIM, st=−0.09

w^RM, st=0.38

Итак, мы видим картину, прямо противоположную той, что видели ранее. Теперь модуль коэффициента  |w^LSTAT, st|=0.45 будет выше, чем модуль коэффициента |w^RM, st|=0.38. Значит, процент низкостатусного населения оказывает большее влияние на значение стоимости жилья, чем количество комнат.

Однако теперь интерпретировать сами коэффициенты в тех же измерениях у нас не получится.

Сделаем важный вывод ↓

Для того чтобы проинтерпретировать оценки коэффициентов линейной регрессии (понять, каков будет прирост целевой переменной при изменении фактора на 1 условную единицу), нам достаточно построить линейную регрессию в обычном виде без стандартизации и получить обычный вектор w⃗ ^.

Однако, чтобы корректно говорить о том, какой фактор оказывает на прогноз большее влияние, необходимо рассматривать стандартизированную оценку вектора коэффициентов w⃗ ^st.

Давайте поближе взглянем на матрицу Грама для стандартизированных факторов:

In [27]:
# матрица Грама
A_st.T @ A_st

,CHAS,LSTAT,CRIM,RM
CHAS,1.000000,-0.053929,-0.055892,0.091251
LSTAT,-0.053929,1.000000,0.455621,-0.613808
CRIM,-0.055892,0.455621,1.000000,-0.219247
RM,0.091251,-0.613808,-0.219247,1.000000


На самом деле мы с вами только что вычислили матрицу выборочных корреляций наших исходных факторов. Мы уже сталкивались с ней много раз в разделах по разведывательному анализу данных и машинному обучению, правда, вычисляли её мы с помощью функции Pandas, а теперь научились делать это вручную.

Примечание. Матрицу корреляций можно получить только в том случае, если производить стандартизацию признаков как векторы (делить на длину центрированного вектора x⃗ st). Другие способы стандартизации/нормализации признаков не превращают матрицу Грама в матрицу корреляций.

### Задание 4.3

2 points possible (graded)

Стандартизируйте вектор x⃗ =(12,8)T, приведя его к единичной длине. В качестве ответа введите координаты полученного вектора. Ответ округлите до третьего знака после точки-разделителя.

In [29]:
x1 = np.array([12, 8])
x1_cent = x1 - x1.mean()
x1_st = x1_cent/np.linalg.norm(x1_cent)
print(x1_st)
## [ 0.70710678 -0.70710678]
print(x1_st.round(3))
## [ 0.707 -0.707]



[ 0.70710678 -0.70710678]
[ 0.707 -0.707]


### Задание 4.7

1 point possible (graded)

Вычислите коэффициент корреляции между векторами  v⃗ =(5,1,2)T  и  u⃗ =(4,2,8)T .

Ответ округлите до двух знаков после точки-разделителя.

## 0.05

In [31]:
v = np.array([5, 1, 2]) 
u = np.array([4, 2, 8])

v_cent = v - v.mean()
u_cent = u - u.mean()

v_st = v_cent/np.linalg.norm(v_cent)
u_st = u_cent/np.linalg.norm(u_cent)    

cor = v_st @ u_st
print(cor.round(2))
## 0.05





0.05


### Задание 4.8

3 points possible (graded)
Составьте корреляционную матрицу для системы векторов:

x⃗ 1=(5.1,1.8,2.1,10.3,12.1,12.6)T 

x⃗ 2=(10.2,3.7,4.1,20.5,24.2,24.1)T 

x⃗ 3=(2.5,0.9,1.1,5.1,6.1,6.3)T 

Для расчёта используйте библиотеку NumPy или Pandas.

In [36]:
x1 = np.array([5.1, 1.8, 2.1, 10.3, 12.1, 12.6])
x2 = np.array([10.2, 3.7, 4.1, 20.5, 24.2, 24.1])
x3 = np.array([2.5, 0.9, 1.1, 5.1, 6.1, 6.3])

A = np.column_stack((x1, x2, x3))

cor_matrix = A.T @ A / (len(A) - 1)
print(cor_matrix)

cor_matrix_pd = pd.DataFrame(cor_matrix, index=['x1', 'x2', 'x3'], columns=['x1', 'x2', 'x3'])
print(cor_matrix_pd)



print(f'Ранг матрицы A: {np.linalg.matrix_rank(A)}')
print(f'Ранг матрицы cor_matrix: {np.linalg.matrix_rank(cor_matrix)}')
print(f'Ранг матрицы cor_matrix_pd: {np.linalg.matrix_rank(cor_matrix_pd)}')


print(f'Определитель корреляционной матрицы: {np.linalg.det(cor_matrix).round(7)}')
print(f'Определитель корреляционной матрицы pd: {np.linalg.det(cor_matrix_pd).round(7)}')




[[ 88.984 174.984  44.48 ]
 [174.984 344.248  87.468]
 [ 44.48   87.468  22.236]]
         x1       x2      x3
x1   88.984  174.984  44.480
x2  174.984  344.248  87.468
x3   44.480   87.468  22.236
Ранг матрицы A: 3
Ранг матрицы cor_matrix: 3
Ранг матрицы cor_matrix_pd: 3
Определитель корреляционной матрицы: 0.0262918
Определитель корреляционной матрицы pd: 0.0262918


In [39]:
# Эталонное решение
x_1 = np.array([5.1, 1.8, 2.1, 10.3, 12.1, 12.6])
x_2 = np.array([10.2, 3.7, 4.1, 20.5, 24.2, 24.1])
x_3 = np.array([2.5, 0.9, 1.1, 5.1, 6.1, 6.3])
C = np.corrcoef([x_1, x_2, x_3])
print('Rank:', np.linalg.matrix_rank(C))
print('Determinant: {:.7f}'.format(np.linalg.det(C)))
print(C)


Rank: 3
Determinant: 0.0000005
[[1.         0.99925473 0.99983661]
 [0.99925473 1.         0.99906626]
 [0.99983661 0.99906626 1.        ]]


Подробное объяснение различий

1. Численная точность и округление
np.corrcoef и pandas.DataFrame.corr используют немного разные подходы к вычислению корреляции. Хотя оба метода реализуют корреляцию Пирсона, Pandas может применять дополнительные округления или оптимизации, которые влияют на конечный результат.

В моём коде Pandas округлил корреляционные коэффициенты до 6 знаков после запятой (например, 0.999905 вместо 0.99990524). Это округление, хотя и кажется незначительным, сильно влияет на определитель, потому что корреляционная матрица почти вырожденная (коэффициенты очень близки к 1).

2. Чувствительность определителя к малым изменениям
Корреляционная матрица в данном случае почти вырожденная, так как все коэффициенты корреляции очень близки к 1. Определитель такой матрицы очень мал, и даже небольшие изменения в значениях (например, с 0.99990524 до 0.999905) могут изменить определитель в разы:

Эталонная матрица (с np.corrcoef): определитель ( 0.0000005 ),

Моя матрица (с pandas.DataFrame.corr): определитель ( 0.0000001 ).



### Задание 6.1
1 point possible (graded)
Построена модель полиномиальной регрессии следующего вида:

$y=10.4+8⋅x_{1}+0.5⋅x_{2}+3⋅x^{2}_{1}+0.4⋅x^{2}_{2}+0⋅x_{1}x_{2} $

Поступило новое наблюдение, которое характеризуется вектором  $x_{new}=(x_{1new},x_{2new})^{T}=(1,4)^{T}$ .

Сделайте прогноз целевой переменной с помощью полученной полиномиальной регрессии. Ответ округлите до первого знака после точки-разделителя.

In [40]:
import numpy as np

# Новое наблюдение
x_new = np.array([1, 4])  # x_1 = 1, x_2 = 4

# Коэффициенты модели
beta_0 = 10.4  # свободный член
beta_1 = 8.0   # коэффициент при x_1
beta_2 = 0.5   # коэффициент при x_2
beta_11 = 3.0  # коэффициент при x_1^2
beta_22 = 0.4  # коэффициент при x_2^2
beta_12 = 0.0  # коэффициент при x_1 * x_2

# Вычисляем прогноз
y_pred = (beta_0 +
          beta_1 * x_new[0] +
          beta_2 * x_new[1] +
          beta_11 * (x_new[0]**2) +
          beta_22 * (x_new[1]**2) +
          beta_12 * (x_new[0] * x_new[1]))

print("Прогноз целевой переменной:", y_pred)
print("Округлённый до первого знака после точки:", round(y_pred, 1))

Прогноз целевой переменной: 29.799999999999997
Округлённый до первого знака после точки: 29.8


### Задание 6.4
3 points possible (graded)
С помощью классического МНК найдите коэффициенты полиномиальной регрессии, если используется полином второй степени и задан фактор x⃗  и целевая переменная y⃗ .
![alt text](2025-04-04_19-32-34.png)

In [41]:
import numpy as np

# Input data
x_vec = np.array([1, 3, -2, 9])
y_vec = np.array([3, 7, -5, 21])

# Construct the design matrix X
# Column of ones for w0
col_w0 = np.ones_like(x_vec)
# Column for w1 (original x)
col_w1 = x_vec
# Column for w2 (x squared)
col_w2 = x_vec**2

# Stack the columns to form X
X = np.column_stack((col_w0, col_w1, col_w2))

print("Design Matrix X:")
print(X)
print("\nTarget vector y:")
print(y_vec)

# Calculate X transpose (XT)
XT = X.T
print("\nX Transpose (XT):")
print(XT)

# Calculate XT * X
XTX = XT @ X
print("\nXT * X:")
print(XTX)

# Calculate the inverse of (XT * X)
try:
    XTX_inv = np.linalg.inv(XTX)
    print("\nInverse of (XT * X):")
    print(XTX_inv)
except np.linalg.LinAlgError:
    print("\nError: Matrix XTX is singular, cannot compute inverse directly.")
    # Handle the case of a singular matrix if necessary (e.g., use pseudo-inverse)
    # For this problem, it should be invertible.
    exit()

# Calculate XT * y
XTy = XT @ y_vec
print("\nXT * y:")
print(XTy)

# Calculate the coefficient vector w_hat = (XTX)^(-1) * (XTy)
w_hat = XTX_inv @ XTy

print("\nCalculated coefficient vector w_hat (before rounding):")
print(w_hat)

# Extract and round the coefficients to one decimal place
w0_hat = round(w_hat[0], 1)
w1_hat = round(w_hat[1], 1)
w2_hat = round(w_hat[2], 1)

print("\nRounded coefficients:")
print(f"w0_hat = {w0_hat}")
print(f"w1_hat = {w1_hat}")
print(f"w2_hat = {w2_hat}")

Design Matrix X:
[[ 1  1  1]
 [ 1  3  9]
 [ 1 -2  4]
 [ 1  9 81]]

Target vector y:
[ 3  7 -5 21]

X Transpose (XT):
[[ 1  1  1  1]
 [ 1  3 -2  9]
 [ 1  9  4 81]]

XT * X:
[[   4   11   95]
 [  11   95  749]
 [  95  749 6659]]

Inverse of (XT * X):
[[ 0.37943533 -0.01109627 -0.00416508]
 [-0.01109627  0.0933221  -0.01033851]
 [-0.00416508 -0.01033851  0.00137246]]

XT * y:
[  26  223 1747]

Calculated coefficient vector w_hat (before rounding):
[ 0.11446013  2.46095638 -0.01608801]

Rounded coefficients:
w0_hat = 0.1
w1_hat = 2.5
w2_hat = -0.0


### Задание 7.4

4 points possible (graded)

Вычислите коэффициенты линейной регрессии с L2-регуляризацией, используя аналитическую формулу Тихонова, если:

x1 = np.array([5, 9, 4, 3, 5])

x2 = np.array([15, 18, 18, 19, 19])

x3 = np.array([7, 6, 7, 7, 7])

y = np.array([24,22, 35, 33, 36])

Коэффициент регуляризации α=1.

В качестве ответа приведите значения полученных коэффициентов линейной регрессии, округлив их до второго знака после точки-разделителя.

[-0.09 -1.71  1.91  0.73]

In [47]:
## Расчет коэффициенты линейной регрессии 
# с L2-регуляризацией, используя аналитическую формулу Тихонова

# Наблюдения
x1 = np.array([5, 9, 4, 3, 5])
x2 = np.array([15, 18, 18, 19, 19])
x3 = np.array([7, 6, 7, 7, 7])

# Целевая переменная
y = np.array([24,22, 35, 33, 36])

# Матрица наблюдений
A = np.column_stack((x1, x2, x3))

# Добавляем столбец из единиц для свободного члена
A = np.column_stack((np.ones(len(x1)), A))

# Коэффициент регуляризации 
alpha = 1

# Количество строк в матрице A.T
k = len(A.T)

# Вычисляем вектор коэффициентов линейной регрессии
w_hat = np.linalg.inv(A.T @ A + alpha * np.eye(k)) @ A.T @ y

print(w_hat.round(2))

## [-0.09 -1.71  1.91  0.73]


[-0.09 -1.71  1.91  0.73]
